# Create fits with dark-matter spectrum

The following shows how to use the class ```dmtable``` to generate a fits table compatible(?) with the ```GModelSpectralTable``` class from ```gammalib```. The fits table is comprised by a ```GNDarray``` where the dimensions refers to the parameters of the model. In our case, the parameters refer to:

1. Mass of the dark matter candidate
2. Channel of annihilation or decay

With this, the ```GModelSpectralTable``` class get the spectrum for different values of the parameters via interpolation. Additionally, you can specify an overall normalization parameter to take into account for extra parameters, like the astrophysical factors or the annihilation cross-section $\langle\sigma_{\chi}v\rangle$ or the lifetime of the candidate $\tau_{\chi}$. This is important because you can create a fits table for a family of targets with similar properties, while the astrophysical factor is the only parameter that changes among all the targets in the sample (for example dwarf spheroidal galaxies). At this moment, the channel parameter is fixed.

The ```dmtable``` take a ```dmspectrum``` instance as one of its parameters, wo you need to create before an instance. The advantage of this approach is that you can change values of the ```dmspectrum``` object via the methods ```hasEW``` and ```process``` to whether to include or not EW corrections in the spectrum, or change between the two processes available at this moment.

In [1]:
from ctaAnalysis.dmspectrum.dmspectra import dmspectrum
from ctaAnalysis.dmspectrum.dmflux_table import dmtable

## Input parameters

The, I define some global parameters of the dark matter candidate (mass \[GeV\], channel, redshift and EBL attenuation model), and the energy range where I compute the spectra. I can also define the process via the ```process``` keyword argument, but I am going with the default here. Both, mass and channel are used to create the ```dmspec``` instance.

In [2]:
mass     = 1.e+3
emin     = 30.
emax     = 100.
channel  = 'Tau'
z        = 0.018
eblmodel = 'dominguez'
epoints  = 300

dmspec = dmspectrum(mass, emin, emax, channel, z, eblmod=eblmodel, epoints=epoints)

## Create ```dmtable``` object

Now, I set the values of the arguments for the ```dmtable``` class. The parameters of the class are:

- ```srcname``` &#10132; Name of the target or family
- ```mmin``` &#10132; Min value of mass
- ```mmax``` &#10132; Mac value of mass
- ```mpoints``` &#10132; Number of points to compute between ```mmin``` and ```mmax``` (inclusive)
- ```dminterp``` &#10132; Object of ```dmspectrum``` class
- ```delta``` &#10132; String that indicates if candidate is a Majorana or Dirac particle (default Majorana)
- ```sigmav``` &#10132; Value of annihilation cross-section (default $3\times10^{-26}~\text{cm}^3~\text{s}^{-1}$)
- ```jfactor``` &#10132; Value of astrophysical J factor (default $1\times10^{19}\text{GeV}^2~\text{cm}^{-5}$)
- ```lifetime``` &#10132; Value of decay lifetime (default $1\times10^{30}~\text{s}$)
- ```dfactor``` &#10132; Value of astrophysical J factor (default $1\times10^{19}~\text{GeV}~\text{cm}^{-2}$)

In [3]:
srcname = 'Toy'
sigmav  = 3.6e-26
jfactor = 1.e+19
mmin    = 100.
mmax    = 1.e+5
mpoints = 200

spectral = dmtable(srcname, mmin, mmax, mpoints, dmspec)

Below, I show how to access to the different available methods or ```dmtable```. For example, I can access to the table model, but at this moment shows a ```None```. This is because I have not execute the method ```create_modeltable```.

In [4]:
print(len(spectral.masses))

200


In [5]:
print(spectral.tablemodel)

None


In [6]:
print(spectral.mmax)

100000.0


In [7]:
print(spectral.mmin)

100.0


In [8]:
print(spectral.jfactor)

1e+19


In [9]:
print(spectral.sigmav)

3.6e-26


In [10]:
print(spectral.delta)

Majorana


I can access to the channels used to compute the spectrum. The channels depends in the property ```hasEW```. In this case, by default, the ```dmspectrum``` object include EW corrections (```hasEW = True```). As you can guess, the channels showed here are taken from the ```PPPC4DMID``` project.

In [11]:
print(spectral.allowed_channels)

('eL', 'eR', 'e', 'MuL', 'MuR', 'Mu', 'TauL', 'TauR', 'Tau', 'q', 'c', 'b', 't', 'WL', 'WT', 'W', 'ZL', 'ZT', 'Z', 'g', 'Gamma', 'h', 'Nue', 'NuMu', 'NuTau', 'Ve', 'VMu', 'VTau')


Ok, perfect. Now, I don't want to include EW corrections so I pass ```False``` to the property ```hasEW```. When doing this, ```dmtable``` also update the list of available channels.

In [12]:
spectral.hasEW = False

In [13]:
print(spectral.allowed_channels)

('e', 'Mu', 'Tau', 'q', 'c', 'b', 't', 'W', 'Z', 'g')


Also, I can change between processes. For example, here I indicate to ```dmtable``` change the process to decay of dark matter, but probably I don't want to use the lifetime and dfactor specified by default, so I can pass the values of both parameters. The method ```process``` updates the value of lifetime and dfactor too. For annihilation you can do exactly the same. The order of the parameters in the list or tuple is:

1. process (anna or decay)
2. astrophysical factor
3. $\langle\sigma_{\chi}v\rangle$ or $\tau_{\chi}$

In [14]:
spectral.process = ['decay', 1.e+20, 1.e+35]
print(spectral.lifetime)

1e+35


In [15]:
spectral.process = ['anna', 1.e+18, 2.4e-26]
print(spectral.sigmav)

2.4e-26


## Creating the fits table

Finally, we can create the ```GModelSpectralTable``` object and save the fits table with the spectrum. In this case, I want to compute the table for 200 different values of masses, 10 available channels (no EW), and 300 values of energy. So, it takes some minutes to create the table.

In [16]:
spectral.create_modeltable()

200it [08:25,  2.53s/it]


We can access to the table and take a look to the classname and the contents of the spectral model.

In [17]:
table = spectral.tablemodel

In [18]:
print(table.classname())

GModelSpectralTable


In [19]:
print(table)

=== GModelSpectralTable ===
 Table file ................: 
 Number of parameters ......: 3
  Normalization ............: 9.54929658551372e-17 +/- 0 [0,1e+60]  (free,scale=1,gradient)
  Mass .....................: 100 GeV (fixed,scale=1)
  Channel ..................: 2  (fixed,scale=1)
 Mass values ...............: 200 [100, 100000] (logarithmic)
 Channel values ............: 10 [0, 9] (linear)
 Energies ..................: 300 [30 GeV, 100 GeV]
 Spectra array dimension ...: 3
 Number of spectra .........: 2000
 Number of spectral bins ...: 300


Ok, now, we can save the fits model using the method ```save```. The name of the fits file is formed with the process used to compute the spectrum, plus the name of the target and an whether or not EW corrections are included (1 or 0). The prefix of the file is ```DMModel```

In [20]:
spectral.save()